# **Project 2- DataScience Projects**

# **DtaScience Project with Netflix Dataset**

Objective

1.Importing Libraries
2.Loading and Exploring Data
3.Data Visulization
4.Text Preprocessing
5.Vectorization
6.Similarity Calculation
7.Training a Model
8.Evaluating Model Performance
9.Making Predictions

### **Import Library**

In [2]:
import pandas as pd
import numpy as np

In [3]:
import seaborn as sns
import matplotlib.pyplot as pllt

In [4]:
!pip install neattext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 5.6 MB/s eta 0:00:00


In [5]:
import neattext.functions as nfx

In [6]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.pipeline import Pipeline

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model  import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier

In [53]:
import matplotlib.pyplot as plt


**Load Dataset**

In [9]:
df = pd.read_csv("/content/drive/MyDrive/8. Netflix Dataset.csv")

In [ ]:
df.head()

,Show_Id,Category,Title,Director,Cast,Country,Release_Date,Rating,Duration,Type,Description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,07:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",PG-13,123 min,Dramas,A brilliant group of students become card-coun...


### **Shape of dataset**

In [10]:
df.shape

(7789, 11)

### **Datatypes**

In [11]:
df.dtypes

Show_Id         object
Category        object
Title           object
Director        object
Cast            object
Country         object
Release_Date    object
Rating          object
Duration        object
Type            object
Description     object
dtype: object

### **Columns**

In [12]:
df.columns

Index(['Show_Id', 'Category', 'Title', 'Director', 'Cast', 'Country',
       'Release_Date', 'Rating', 'Duration', 'Type', 'Description'],
      dtype='object')

### **Clean Dataset**

In [ ]:
dir(nfx)

### **Data Visulization**

In [ ]:
# Scatter plot
sns.scatterplot(data=df, x='Category', y='Title')
plt.title('Scatter Plot')
plt.show()

# Histogram
sns.histplot(data=df, x='column', bins=10)
plt.title('Histogram')
plt.show()

# Bar plot
sns.barplot(data=df, x='column', y='count')
plt.title('Bar Plot')
plt.show()

# Box plot
sns.boxplot(data=df, x='column', y='value')
plt.title('Box Plot')
plt.show()

# Heatmap
correlation_matrix = df.corr()
sns.heatmap(data=correlation_matrix, annot=True)
plt.title('Heatmap')
plt.show()

In [14]:
df['Title'].str.lower()

0                                            3%
1                                         07:19
2                                         23:59
3                                             9
4                                            21
                         ...                   
7784                                       zozo
7785                                     zubaan
7786                          zulu man in japan
7787                      zumbo's just desserts
7788    zz top: that little ol' band from texas
Name: Title, Length: 7789, dtype: object

In [15]:
df['Movie_Title_clean'] = df['Title'].str.lower().apply(lambda x:nfx.remove_special_characters(x))

In [16]:
df['Movie_Title_clean'] = df['Movie_Title_clean'].apply(lambda x:nfx.remove_stopwords(x))

In [17]:
df[['Title','Movie_Title_clean']]

,Title,Movie_Title_clean
0,3%,3
1,07:19,0719
2,23:59,2359
3,9,9
4,21,21
...,...,...
7784,Zozo,zozo
7785,Zubaan,zubaan
7786,Zulu Man in Japan,zulu man japan
7787,Zumbo's Just Desserts,zumbos desserts


### **Recommendation System Using Movie Title**

In [25]:
#Convert text to vector Features
cv = CountVectorizer()

In [26]:
vectorized_text = cv.fit_transform(df['Movie_Title_clean'])

In [27]:
from pandas._libs.tslibs import vectorized
vectorized_text.toarray

<bound method _cs_matrix.toarray of <7789x8167 sparse matrix of type '<class 'numpy.int64'>'
	with 17347 stored elements in Compressed Sparse Row format>>

In [28]:
vectorized_text.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [ ]:
#Vocab
cv.vocabulary_

In [29]:
#Get features
cv.get_feature_names_out()

array(['009', '0719', '10', ..., 'zulu', 'zumbos', 'zz'], dtype=object)

In [30]:
from sklearn.metrics.pairwise import cosine_similarity

In [35]:
#Cosine matrix
cosine_mat = cosine_similarity(vectorized_text)

In [36]:
cosine_mat

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [37]:
# Indices of movie titles
movie_indices = pd.Series(df.index,index=df["Movie_Title_clean"]).drop_duplicates()


In [38]:
movie_indices

Movie_Title_clean
3                             0
0719                          1
2359                          2
9                             3
21                            4
                           ... 
zozo                       7784
zubaan                     7785
zulu man japan             7786
zumbos desserts            7787
zz little ol band texas    7788
Length: 7789, dtype: int64

In [40]:
# Get movies index from indices
idx = movie_indices['zozo']
idx

7784

### **Split Dataset**

In [41]:
Xfeatures = df['Movie_Title_clean']
ylabels = df['Rating']

In [42]:
#Splite dataset
x_train,x_test,y_train,y_test = train_test_split(Xfeatures,ylabels,test_size=0.3,random_state=42)

### **Modeling**

In [48]:
from sklearn.metrics import accuracy_score

In [49]:
model = LogisticRegression()

# Training the model
model.fit(X_train, y_train)

# Making predictions on the testing data
y_pred = model.predict(X_test)

# Evaluating the model's performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.0


### **Prediction**

In [51]:
#predictions on the dataset
y_pred = model.predict(X)

print("Predictions:", y_pred)

Predictions: [0 0 0 1]


### **Explaination**

Importing necessary libraries for data manipulation, visualization, text processing, and machine learning.

Loading a dataset using pandas and displaying the first few rows, shape, data types, and column names.

Preprocessing the text data in the 'Title' column by converting it to lowercase and applying functions from the neattext library to remove special characters and stopwords.

Converting the preprocessed text into numerical vector features using CountVectorizer.

Calculating the cosine similarity matrix based on the vectorized text.

Creating a mapping of movie titles to their indices.

Splitting the dataset into training and testing sets.

Training a logistic regression model on the training data.

Making predictions on the testing data and evaluating the model's performance using accuracy score.

Making predictions on the entire dataset.

The code essentially performs text preprocessing, vectorization, model training, and prediction tasks on a dataset.